<a href="https://colab.research.google.com/github/HIRO76/charts/blob/gh-pages/email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Email送信（gmail)

In [0]:
import smtplib  # SMTPのインターネット上のホストにメールを送るために使用
from getpass import getpass  # エコーなしでユーザーにパスワードを入力させるプロンプト
import time  # 時刻に関するさまざまな関数を提供

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# アカウント名、パスワード格納
LOGIN_USERNAME = None
LOGIN_PASSWORD = None

def send_mail(recipient, address, password, subject, text):
    # SMTPサーバーへ接続
    smtpserver = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)  
    # 拡張機能の確認
    smtpserver.ehlo()  
    # tlsでメール通信経路暗号化
    smtpserver.starttls()  
    # 拡張機能の確認
    smtpserver.ehlo()  
    # SMTPサーバーにログイン
    smtpserver.login(address, password)  
    # メールのヘッダー作成
    header = "To: " + recipient + "\n" + "From: " + address
    header = header + "\n" + "Subject: " + subject + "\n"
    # ヘッダーと本文を結合
    message = header + "\n" + text
    # メール送信
    smtpserver.sendmail(address, recipient, message)
    # SMTPサーバー切断
    smtpserver.close() 

if __name__ == "__main__":
    # 差出人入力
    email_address_to = input("Email address to : ")
    # 宛先入力
    if not LOGIN_USERNAME:
        email_address_from = input("Email address from : ")
    else:
        LOGIN_USERNAME
    # パスワード入力
    if not LOGIN_PASSWORD:
        email_password = getpass("Email password : ")
    else:
        LOGIN_PASSWORD
    # 件名
    subject = "test mail"
    # 本文
    text = "Hello : " + time.strftime("%I:%M:%S")
    # メール送信関数呼び出し
    send_mail(emal_address_to, email_address_from, email_password, subject, text)


# ファイルを添付してメール送信

In [0]:
# マルチパート、添付ファイルを取り扱うためのモジュール
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
from os.path import basename

def send_file(to_email, from_email, password, path, subject, message):
    # MIME作成
    msg = MIMEMultipart()
    msg["Subject"] = subject
    msg["To"] = to_email
    msg["From"] = from_email
    msg.attach(MIMEText(message))

    # ファイル添付
    with open(path, "rb") as f:
        part = MIMEApplication(f.read(), Name=basename(path))

    # ファイル名設定
    part["Content-Disposition"] = "attachment: filename=" basename(path)
    msg.attach(part)

    # 送信処理
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(from_email, password)
    server.send_message(msg)
    server.quit()

# 情報入力、実行
if __name__ == "__main__":
    to_email = "000@gmail.com"
    from_email = "XXX@gmail.com"
    password = "password"
    path = "sample.csv"
    subject = "file data"
    message = "sample file attached"
    send_file(to_email, from_email, password, path, subject, message)


# 受信メールチェック

In [0]:
# IMAP4 サーバへの接続
import imaplib

SERVER_NAME = "imap.gmail.com"
USERNAME = "OOO@gmail.com"
PASSWORD = "password"

mail = imaplib.IMAP4_SSL(SERVER_NAME)
mail.login(USERNAME, PASSWORD)
mail.select("Inbox")

# search()は検索結果結果をタプルで戻す
# b_mlistはメールのスペース区切り連番数値が戻り値として入る
st, b_mlist = mail.search(None, "UNSEEN")

if (st == "OK"):
    # メールの連番数値からsplit()でスペース区切りのリストを生成
    mlist = b_mlist[0].decode().split()
    # リストの要素数が未読メール数となる
    mcount = len(mlist)
    if (mcount > 0):
        print(f"You got {str(mcount)} mail")
    else:
        print("No new mail")
else:
    print("Can't get mail status")

mail.close()
mail.logout()

# メール添付ファイルの保存

In [0]:
import imaplib
import email.parser
import email.header
import os
import os.path

SERVER_NAME = "imap.gmail.com"
USER_NAME = "hoge@gmail.com"
PASSWORD = "hogehoge"
MBOX = "INBOX"
JSN_DIR = "./jsn"

# mimeエンコードされたheaderをデコードしstr文字列を返す
# MIMEエンコードヘッダーをデコードして文字列を取得
def decode_mime_header(s0):
    return "".join(str(s, c or "ascii") if is instance(s, (bytes,)) else s for s, c in email.header.decode_header(s0))

# 引数「obj」メールオブジェクトの添付ファイル有無を判定
# objが添付ファイルかどうかを確認
def is_attachment(obj):
    s = obj["Content-Disposition"]
    return s and s.startswith("attachment")

# get_jsn_each()はget_jsn()から呼び出され、引数「e」メールオブジェクトの
# 添付ファイル(json)を引数「d_jsn」で指定されたディレクトリに保存
def get_json_each(d_jsn, e):
    if e.is_multipart():
        for p in walk():
            if not p.multipart() and is_attachment(p) and p.get_content_maintype() == "application":
                fname0 = p.get_filename()
                if fname0:
                    fname = decode_mime_header(fname0)
                    try: 
                        with open(os.path.join(d_jsn, fname), "wb")as f:
                            f.write(p.get_payload(None, True))
                    except:
                        print("cannot save" + fname)
                    print(fname)
                else:
                    print("cannot get filename")

# すべてのメールの添付ファイルをmboxに保存
# 'UNSEEN'を指定すると未読メールオブジェクトの添付ファイルを保存
def get_jsn(con, mbox, d_jsn):
    con.select(mbox)
    p = email.parser.BytesParser()
    _type, _data = con.search(None, "ALL")
    # _type, _data = con.search(None, "UNSEEN")

    for num in _data[0].split():
        type, data = con.fetch(num, "(RCF822)")
        get_json_each(d_jsn, p.parsebytes(data[0][1]))

if __name__ == "__main__":
    if not os.path.isdir(JSN_DIR):
        os.mkdir(JSN_DIR)
    con = imaplib.IMAP4_SSL(SERVER_NAME)
    con.login(USER_NAME, PASSWORD)
    get_jsn(con, MBOX, JSN_DIR)
    con.close()
    con.logout()
